# 2023/06/24 시행 문제 

## 작업형 1유형

### 문제1
- 각 구급 보고서별 출동시각과 신고시각의 차이를 '소요시간' 컬럼으로 만들어 초(sec)단위로 구하고
- 소방소별 소요시간 평균을 오름차순으로 정렬 
- 3번째로 작은 소요시간의 값과 소방서명을 출력


In [105]:
import pandas as pd

data = pd.read_csv('https://raw.githubusercontent.com/JEunJin/Bigdata_python/master/bigdata_csvfile/report_data.csv')
display(data.head()) # 결측치 없음 

# 신고일자 + 신고시각 / 출동일자 + 출동시각 을 datatime형으로 변경  , 331 > 000331  0채워서 6자리로 만들어야함 : str.zfill(6)
data['신고시각_dt'] = pd.to_datetime(data['신고일자'].astype(str)+data['신고시각'].astype(str).str.zfill(6))
data['출동시각_dt'] = pd.to_datetime(data['출동일자'].astype(str)+data['출동시각'].astype(str).str.zfill(6))
data['소요시간'] = (data['출동시각_dt'] - data['신고시각_dt']).dt.total_seconds() # 일자 시간 차이계산시 DateTime.dt.totl_sconds()

#display(data.head()) # 결측치 없음 
# 소방소별 소요시간 평균 = groupby, index 활성화를 위해 .reset_index()
data = data.groupby(['소방서명'])['소요시간'].mean().reset_index()  
data.sort_values('소요시간',inplace=True, ascending=True)
display(data.head(3))
#print(data)
print(data.iloc[2]['소방서명'] ,data.iloc[2]['소요시간'] )

,구급보고서번호,소방서명,환자증상구분명1,신고일자,신고시각,출동일자,출동시각,환자직업명,사고발생장소
0,20211122103M01526,중랑소방서,심계항진,20210330,104800,20210330,105042,무직,집
1,20211111105M00771,노원소방서,복통,20210412,225600,20210412,225827,기타,집
2,20211115104M00864,서초소방서,심계항진,20210303,194100,20210303,194500,직장인,상업시설
3,20211113107M01869,마포소방서,고열,20211225,235400,20211226,331,유아,집
4,20211118102M05820,강동소방서,고열,20211019,12100,20211019,12322,무직,집


,소방서명,소요시간
13,서초소방서,164.883495
12,서대문소방서,165.056604
21,종로소방서,175.500000


종로소방서 175.5


### 문제 2
- 학교 세부유형이 일반중학교인 학교들 중 일반중학교 숫자가 2번째로 많은 시도의 일반중학교만 필터링
- 해당 시도의 한 명당 맡은 학생수가 가장 많은 학교를 찾아 , 해당 학교의 교원 수를 출력
- 단, 값이 0 or NaN 인경우 없는 것으로 판단하여 연산에서 제외

In [106]:
import pandas as pd

data = pd.read_csv('https://raw.githubusercontent.com/JEunJin/Bigdata_python/master/bigdata_csvfile/class_data.csv')
#display(data.info()) #결측치는 없음
# 일반 중학교들만 필터링
data = data[data['학교세부유형']=='일반중학교']
# 시도별 중학교 수가 2번째로 많은곳만 필터링 (=서울)
#display(data['시도'].value_counts())# '시도' = 서울
data = data[data['시도']=='서울']
#
data['인당_맡은_학생수'] = data['일반학급_학생수_계']/data['교원수_총계_계']
# 인당 맡은 학생수에 결측치 1개 발생 , 아마 하나가 0 이였음
#display(data.info())
data = data.dropna()
#display(data.info())
data = data.sort_values('인당_맡은_학생수',ascending=False)
display(data) # 단국대학교사범대학부속중학교 
# 해당학교의 교원수 
#print(data[data['학교명']=='단국대학교사범대학부속중학교']['교원수_총계_계'].values)
print(data['교원수_총계_계'].iloc[0])

,학교명,시도,학교세부유형,일반학급_학생수_계,교원수_총계_계,인당_맡은_학생수
524,단국대학교사범대학부속중학교,서울,일반중학교,606,33,18.363636
444,한영중학교,서울,일반중학교,742,43,17.255814
532,진선여자중학교,서울,일반중학교,997,58,17.189655
441,배재중학교,서울,일반중학교,868,51,17.019608
526,휘문중학교,서울,일반중학교,814,48,16.958333
...,...,...,...,...,...,...
398,환일중학교,서울,일반중학교,151,27,5.592593
438,동북중학교,서울,일반중학교,233,43,5.418605
256,정화여자중학교,서울,일반중학교,125,24,5.208333
453,경서중학교,서울,일반중학교,112,26,4.307692


33


# 문제 3 
- 5대범죄(절도,사기,배임,방화,폭행)의 월별 총발생건수 = 총범죄수 
- 2018년, 2019년 각 분기별 총범죄수 월평균 값 구하기 
- 최대값을 가지는 연도와 분기를 구해라 
- 해당 분기의 최댓값의 사기 범죄가 발생한 월의 사기범죄 발생건수를 출력하시오 
- 123 / 456 / 789 / 10 11 12 각 분기


In [107]:
import pandas as pd

data = pd.read_csv('https://raw.githubusercontent.com/JEunJin/Bigdata_python/master/bigdata_csvfile/crime_data.csv')
# 5대범죄 총 발생 건수 
data['총범죄수'] = data['절도']+data['사기']+data['배임']+data['방화']+data['폭행']
display(data.head())
q_2018_1 = data.iloc[0:3]['총범죄수'].sum()
q_2018_2 = data.iloc[3:6]['총범죄수'].sum()
q_2018_3 = data.iloc[6:9]['총범죄수'].sum()
q_2018_4 = data.iloc[9:12]['총범죄수'].sum()

q_2019_1 = data.iloc[12:15]['총범죄수'].sum()
q_2019_2 = data.iloc[15:18]['총범죄수'].sum()
q_2019_3 = data.iloc[18:21]['총범죄수'].sum()
q_2019_4 = data.iloc[21:]['총범죄수'].sum()

print([q_2018_1,q_2018_2,q_2018_3,q_2018_4,q_2019_1,q_2019_2,q_2019_3,q_2019_4]) # 2019년_2분기(4,5,6)
# 해당 분기 최댓값
max_qt = data.iloc[15:18]
result = max_qt['사기'].max()
print(result)


,범죄분류,절도,사기,배임,방화,폭행,총범죄수
0,2018년_1월,13280,25318,656,143,12186,51583
1,2018년_2월,12504,19159,403,135,12087,44288
2,2018년_3월,14733,24509,471,148,13841,53702
3,2018년_4월,14781,23147,414,135,13973,52450
4,2018년_5월,15988,23584,451,127,14117,54267


[np.int64(149573), np.int64(160387), np.int64(166385), np.int64(151445), np.int64(159412), np.int64(175315), np.int64(175004), np.int64(160747)]
27766


## 문제 2
- 예측변수 General_Health에 대해 ID별 General_Health 값을 예측하여 제출 
- 단 제출 데이터의 컬럼은 ID와 General_Health 두개만 존재 ,
- 퍙가지표는 macro f1 score로 
- 제출파일명 : 6th_test_type2.csv

In [108]:
import pandas as pd

train = pd.read_csv('https://raw.githubusercontent.com/JEunJin/Bigdata_python/master/bigdata_csvfile/test6_type2_train.csv')
test = pd.read_csv('https://raw.githubusercontent.com/JEunJin/Bigdata_python/master/bigdata_csvfile/test6_type2_test.csv')

#display(train.head()) # General_Health 가  target
#train.info() # 결축치 없음 , 모델학습을 위해 라벨 인코딩 필요, ID drop, General_Health 타겟겟

# 라벨인코딩 
from sklearn.preprocessing import LabelEncoder
list_object = ['Checkup','Exercise','Heart_Disease','Skin_Cancer','Depression','Diabetes','Sex','Smoking_History','Other_Cancer','Age_Category','Arthritis']
le = LabelEncoder()
for l in list_object:
    train[l] = le.fit_transform(train[l])
    test[l] = le.fit_transform(test[l])
#test.info()

from sklearn.model_selection import train_test_split
X = train.drop(columns=['ID','General_Health'])
y = train['General_Health']

X_train , X_val , y_train, y_val = train_test_split(X,y,train_size=0.2)
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import f1_score
rfc = RandomForestClassifier()

rfc.fit(X_train,y_train)
y_pred = rfc.predict(X_val)

f1 = f1_score(y_pred, y_val, average='macro') # macro f1 score  == averge = 'macro'

#print(f1) 0.54 정확도 ? 


In [109]:
test.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 4813 entries, 0 to 4812
Data columns (total 19 columns):
 #   Column                        Non-Null Count  Dtype  
---  ------                        --------------  -----  
 0   ID                            4813 non-null   object 
 1   Checkup                       4813 non-null   int64  
 2   Exercise                      4813 non-null   int64  
 3   Heart_Disease                 4813 non-null   int64  
 4   Skin_Cancer                   4813 non-null   int64  
 5   Other_Cancer                  4813 non-null   int64  
 6   Depression                    4813 non-null   int64  
 7   Diabetes                      4813 non-null   int64  
 8   Arthritis                     4813 non-null   int64  
 9   Sex                           4813 non-null   int64  
 10  Age_Category                  4813 non-null   int64  
 11  Height_(cm)                   4813 non-null   float64
 12  Weight_(kg)                   4813 non-null   float64
 13  BMI

In [110]:
X_test = test.drop(columns=['ID'])

y_pred2 = rfc.predict(X_test)

pd.DataFrame({'ID':test['ID'],'General_Health':y_pred2}).to_csv('6th_test_type2.csv',index= False)

a= pd.read_csv('6th_test_type2.csv')
display(a)

,ID,General_Health
0,ID_14547,Excellent
1,ID_26,Excellent
2,ID_14103,Good
3,ID_10946,Excellent
4,ID_1572,Excellent
...,...,...
4808,ID_12991,Excellent
4809,ID_13295,Fair
4810,ID_2428,Fair
4811,ID_12719,Good


## 문제 3 
- 전재_조건: 남:700 / 여 500명 
- 남성 기혼자 비율: 0.3
- 여성 기혼자 비율: 0.2 
= 혼인 여부엘 따른 인구 비율이 동일한가?
- 1. 남, 여 기혼자와 미혼자에 대한 카이제곱 검정 진행 , 카이제곱통계량 출력
- 2. 유의수준 0.05를 기준 , 카이제곱 검정의 결과 p_value 출력
- 3. 검증된 p_value를 바탕으로 '귀무가설_기각' 혹은 '귀무가설_채택'을 출력력

In [111]:
# 카이제곱 통계량
from scipy.stats import chi2_contingency
import pandas as pd
import numpy as np
male = 700
female = 500

male_married = int(male*0.3)
female_married = int(female*0.2)

male_solo = male - male_married
female_solo = female - female_married

matrixs = np.array([[male_married,male_solo],[female_married,female_solo]])
print(matrixs)

chi2,p_val,dof,expected = chi2_contingency(matrixs) # c p d e  모르면 외워  chi2 / p_value / dof / expected
#문제 1
print(chi2) # 14.71
#문제 2
print(p_val)# 0.0001
#문제 3
print('귀무가설 기각')

[[210 490]
 [100 400]]
14.70545228602495
0.00012568242320118222
귀무가설 기각


## 문제 2
- 주어진 데이터 다중 회귀분석 수행 
- 1. age 와 Cholesterol 컬럼으로 weight를 예측하는 선형회귀모형 만들기, age의 회귀계수를 구하시오(소수점 3자리 까지 반올림)
- 2. age가 45이고 , Cholesterol이 67.8일때 문제 1 모델을 기반으로 weight를 예측
- 3. 문제 1 의 유의수준이 0.05일때 Cholesterol 회귀계수의 신뢰구간을 구해라


In [112]:
import pandas as pd

data = pd.read_csv('https://raw.githubusercontent.com/JEunJin/Bigdata_python/master/bigdata_csvfile/cholesterol_data.csv')
#print(data.info())
# 선형회귀모델 만들기 위해 상수항 추가 필요
import statsmodels.api as sm
X = sm.add_constant(data[['age','Cholesterol']])
model = sm.OLS(data['weight'],X) # sm.OLS(예측할 값, 그외 데이터)
result = model.fit()
print(result.summary())
# 문제1
answer1 = result.params['age']
print(round(answer1,3))
# 문제2 
#answer2 = result.params['const']*1 + result.params['age']*45 + result.params['Cholesterol']*67.8  = 같은 의미 
answer2 = result.predict([1,45,67.8])
print(answer2)
# 문제3
answer3 = '0.039~0.125'
print(answer3)


                            OLS Regression Results                            
Dep. Variable:                 weight   R-squared:                       0.044
Model:                            OLS   Adj. R-squared:                  0.038
Method:                 Least Squares   F-statistic:                     7.574
Date:                Tue, 03 Jun 2025   Prob (F-statistic):           0.000608
Time:                        16:17:14   Log-Likelihood:                -1374.9
No. Observations:                 333   AIC:                             2756.
Df Residuals:                     330   BIC:                             2767.
Df Model:                           2                                         
Covariance Type:            nonrobust                                         
                  coef    std err          t      P>|t|      [0.025      0.975]
-------------------------------------------------------------------------------
const          74.8953      4.455     16.813      